In [1]:
import numpy as np
from scipy import sparse

In [2]:
# check the ordering for the sparse diagonal arrays
diags = [-1,0,1]
data = np.array([
    [21,32,-1],
    [11,22,33],
    [-1,12,23]])

sparse.spdiags(data, diags,3,3).todense()

matrix([[11, 12,  0],
        [21, 22, 23],
        [ 0, 32, 33]])

In [6]:
class KdVBase(object):
    """
    Base class for the KdV solver
    """
    
    Nx = 100
    dx_s = 100
    dt_s = 10.
    
    c1 = 1.1
    r10 = 0.
    r01 = 0.
    r20 = 0.
    
    nu_H = 0.
    
    ekdv = False
    nonlinear = True
    nonhydrostatic = True
    
    mu = 1.
    epsilon = 1.
    
    c_im = 0.5 # IMEX only
    
    def __init__(self, **kwargs):
        """
        
        """
        self.__dict__.update(**kwargs)
    
    
    def build_lhs_matrix(self):
        """
        Build the LHS sparse matrix
        """

        diags1 = self.build_linear_diags()

        # Ones down primary diagonal
        diags2 = np.zeros_like(diags1)
        diags2[2,:] = 1.

        cff = self.dt_s*(1+self.c_im)*0.5        
        diags =  diags2 - cff*diags1
        
        #
        self.insert_bcs(diags)

        # Build the sparse matrix
        M = sparse.spdiags(diags, [-2,-1,0,1,2], self.Nx, self.Nx)

        return M, diags

    def build_nonlinear_diags(self, An):
        """
        Build the nonlinear steepening term
        """
        diags = np.zeros((5,self.Nx))
        
        # Add the nonlinear terms
        cff2 = 2*self.epsilon*self.r10*self.c1
        cff3 = 0.5*cff2/self.dx_s
        cff3 *= 0.5
        if self.nonlinear:
            diags[1,:] = diags[1,:] - cff3*An
            diags[3,:] = diags[3,:] + cff3*An

        # extended KdV
        if self.ekdv:
            cff4 = 3*self.epsilon**2*self.r20*self.c1**2
            cff5 = 0.5*cff4/self.dx_s
            An2 = 0.25*np.power(An, 2.)
            diags[1,:] = diags[1,:] - cff5*An2
            diags[3,:] = diags[3,:] + cff5*An2
            
        ## Bottom friction parameterization (Holloway et al, 1997)
        #if self.k_chezy > 0:
        #    cff = -self.k_chezy*self.c1 / self.H**2.
        #    diags[2,:] += cff * np.abs(An)
        
        return diags

    def build_nonlinear_matrix(self, An):
        """
        Build the nonlinear steepening term
        """
        diags = self.build_nonlinear_diags(An)
        
        self.insert_bcs(diags)

        # Build the sparse matrix
        M = sparse.spdiags(diags, [-2,-1,0,1,2], self.Nx, self.Nx)

        return M

    def build_linear_diags(self):
        """
        Build the linear matrices
        """

        diags = np.zeros((5,self.Nx))

        # pressure terms
        diags[1,:] -= (-0.5*self.c1/self.dx_s) * np.ones((self.Nx,)) #i-1
        diags[3,:] -= (+0.5*self.c1/self.dx_s) * np.ones((self.Nx,)) #i+1

        # Constants
        cff1 = 1*self.mu*self.r01
        #cff1 = 0
        dx3 = 1./np.power(self.dx_s,3.)
        
        # Dispersion term (2nd order)
        if self.nonhydrostatic:
            diags[0,:] += -0.5*cff1*dx3 * np.ones((self.Nx,))
            diags[1,:] += (+cff1*dx3) * np.ones((self.Nx,))
            diags[3,:] += (-cff1*dx3) * np.ones((self.Nx,))
            diags[4,:] += 0.5*cff1*dx3 * np.ones((self.Nx,))

        # Dispersion term (4th order)
        #diags[0,:] += -1/8.*cff1*dx3 * np.ones((self.Nx,))
        #diags[1,:] += -1*cff1*dx3 * np.ones((self.Nx,))
        #diags[2,:] += 13/8.*cff1*dx3 * np.ones((self.Nx,))
        #diags[4,:] += -13/8.*cff1*dx3 * np.ones((self.Nx,))
        #diags[5,:] += +1*cff1*dx3 * np.ones((self.Nx,))
        #diags[6,:] += +1/8.*cff1*dx3 * np.ones((self.Nx,))

        ## Add Laplacian diffusion operator
        #nu_H = 1e1
        nu_H = self.nu_H
        dx2 = 1./np.power(self.dx_s,2.)
        # 2nd order
        diags[1,:] += 0.5*nu_H*dx2 * np.ones((self.Nx,))
        diags[2,:] -= 1*(nu_H*dx2) * np.ones((self.Nx,))
        diags[3,:] += 0.5*nu_H*dx2* np.ones((self.Nx,))

        ## 4th order
        #c1 = -1/12.
        #c2 = 16/12.
        #c3 = -30/12.
        #c4 = 16/12.
        #c5 = -1/12.
        #diags[0,:] += c1*nu_H*dx2 * np.ones((self.Nx,))
        #diags[1,:] += c2*nu_H*dx2 * np.ones((self.Nx,))
        #diags[2,:] += c3*nu_H*dx2 * np.ones((self.Nx,))
        #diags[3,:] += c4*nu_H*dx2* np.ones((self.Nx,))
        #diags[4,:] += c5*nu_H*dx2 * np.ones((self.Nx,))

        # Build the sparse matrix
        #M = sparse.spdiags(diags, [-2,-1,0,1,2], self.Nx, self.Nx)

        return diags
    
    def insert_bcs(self, diags):
        # Set the boundary conditions for the diagonal array
        #[-2,-1,0,1,2]
        
        # top row
        diags[3,1] = 0
        diags[4,2] = 0
        
        # second row
        diags[1,0] = 0
        diags[3,2] = 0
        diags[4,3] = 0
        
        # third row
        diags[0,0] = 0
        diags[1,1] = 0
        diags[3,3] = 0
        diags[4,4] = 0
        
        Nx = self.Nx-1
        
        # bottom row
        #diags[1, Nx-1] = 0
        #diags[0,Nx-2] = 0
        
        # Second bottom row
        #diags[1, Nx-2] = 0
        #diags[0, Nx-3] = 0
        #diags[3, Nx] = 0

        return


In [7]:
kdv = KdVBase(Nx=6, r01=1000., r10 = 1e-3, c1=1.1)
lhs, dd = kdv.build_lhs_matrix()
lhs.todense()

matrix([[ 1.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
        [ 0.     ,  1.     ,  0.     ,  0.     ,  0.     ,  0.     ],
        [ 0.     ,  0.     ,  1.     ,  0.     ,  0.     ,  0.     ],
        [ 0.     ,  0.00375, -0.04875,  1.     ,  0.04875, -0.00375],
        [ 0.     ,  0.     ,  0.00375, -0.04875,  1.     ,  0.04875],
        [ 0.     ,  0.     ,  0.     ,  0.00375, -0.04875,  1.     ]])

In [8]:
A = kdv.build_nonlinear_matrix(10*np.ones(7,))
A.todense()

matrix([[ 0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,
          0.0e+00],
        [ 0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,
          0.0e+00],
        [ 0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,
          0.0e+00],
        [ 0.0e+00,  0.0e+00, -5.5e-05,  0.0e+00,  5.5e-05,  0.0e+00,
          0.0e+00],
        [ 0.0e+00,  0.0e+00,  0.0e+00, -5.5e-05,  0.0e+00,  5.5e-05,
          0.0e+00],
        [ 0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,
          0.0e+00],
        [ 0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,  0.0e+00,
          0.0e+00]])

In [147]:
dd

array([[ 0.00375,  0.00375,  0.     ,  0.     ,  0.00375,  0.00375],
       [ 0.     , -0.045  , -0.045  ,  0.     ,  0.     , -0.045  ],
       [ 1.     ,  1.     ,  1.     ,  1.     ,  1.     ,  1.     ],
       [ 0.045  ,  0.     ,  0.     ,  0.045  ,  0.045  ,  0.     ],
       [-0.00375, -0.00375,  0.     ,  0.     , -0.00375, -0.00375]])